In [1]:
# from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os


from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)



In [2]:
load_dotenv()
key = os.getenv('GOOGLE_API_KEY')
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro", temperature=0.1, api_key=key)
llm = Ollama(model="qwen2:1.5b")

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
persist_directory="./chroma_db"

In [3]:
loader = PyPDFLoader("./doc.pdf")
data = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=1)
all_splits = text_splitter.split_documents(data)
len(all_splits)


15

In [4]:
db = Chroma.from_documents(all_splits, gemini_embeddings)

In [5]:
query = "What is The Document is About?"
matching_docs = db.similarity_search(query)
matching_docs

[Document(metadata={'page': 1, 'source': './doc.pdf'}, page_content='A l g o r i t h m s A m a z o n  W e b  S e r v i c e s  \x00 A W S \x00 , C y b e r s e c u r i t y D j a n g o S e l e n i u m A I L L M s\nR A G \x00 R e t r i e v a l - A u g m e n t e d  G e n e r a t i o n ) L a n g C h a i n N g i n x R e s t  A P I D o c k e r\nwww .enhancv .com\ue06dPowered by•\n•\n•\n•'),
 Document(metadata={'page': 1, 'source': './doc.pdf'}, page_content='Demonstrated strong problem-solving skills by addressing challenges during the development and testing phases.\nCERTIFICATIONS\nC y b e r s e c u r i t y  I n t e r n s h i p  \x00  P a l o a l t o  \x00 V i r t u a l )\nP a l o a l t oA W S  A c a d e m y  G r a d u a t e  \x00  A W S  A c a d e m y  C l o u d  \nA r c h i t e c t i n g\nA m a z o n  W e b  S e r v i c e s  \x00 A W S \x00'),
 Document(metadata={'page': 0, 'source': './doc.pdf'}, page_content="As an accomplished Computer Science\xa0enthusiast,\xa0and a passion for technol

In [6]:
vectordb = Chroma.from_documents(
    documents=all_splits, embedding=gemini_embeddings, persist_directory=persist_directory
)

vectordb.persist()

Document Chains in LangChain are a powerful tool that can be used for various purposes.
Efficient Document Processing: Document Chains allow you to process and analyze large amounts of text data efficiently. They provide a structured approach to working with documents, enabling you to retrieve, filter, refine, and rank them based on specific criteria.

Task Decomposition: Document Chains can help you break down complex tasks into smaller, more manageable subtasks. By using different types of Document Chains like Stuff, Refine, Map Reduce, or Map Re-rank, you can perform specific operations on the retrieved documents and obtain more accurate and relevant results.

Improved Accuracy: Document Chains, especially Map Re-rank Chains, can help improve the accuracy of your responses. By running an initial prompt on each document and returning the highest-scoring response, you can prioritize the most reliable and accurate answers.


# Stuff Chain
One way to provide context to a language model is through the stuffing method.

This involves putting all relevant data into the prompt for the LangChain’s StuffDocumentsChain to process.

The advantage of this method is that it only requires one call to the LLM, and the model has access to all the information at once.

However, one downside is that most LLMs can only handle a certain amount of context. For large or multiple documents, stuffing may result in a prompt that exceeds the context limit.

Additionally, this method is only suitable for smaller amounts of data. When working with larger amounts, alternative approaches should be used.

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "What type of document is this"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer


# ADD History To CHAT BOT